# Compile results

In [1]:
import os
import gc
import ijson
import zipfile
from tqdm.notebook import tqdm

In [2]:
import json
import warnings
import itertools
import numpy as np
from numpy import arange

In [3]:
from pNeuma_simulator.results import loader, aggregate, percolate

In [4]:
warnings.filterwarnings("ignore")

In [5]:
path = "./output/"
os.makedirs(path, exist_ok=True)
# Combinatorial configurations
n_veh = 8
scale = 2
l_cars = scale * arange(1, n_veh, 1)
l_moto = scale * arange(0, n_veh - 1, 1)
permutations = list(itertools.product(l_cars, l_moto))
len(permutations)

49

In [6]:
# consider only this last fraction of the simulated data
keep = 1 / 3
# Pseudorandom number generator
seed = 1024
rng = np.random.default_rng(seed)
compile = True

In [7]:
read = False  # not compile
path = "./output/"
os.makedirs(path, exist_ok=True)
if read:
    with open(f"{path}results.json", "r") as openfile:
        results = json.load(openfile)

In [ ]:
path = "./output/"
os.makedirs(path, exist_ok=True)
verbose = True
permutation = (2, 2)
howmany = 1
items = []
with zipfile.ZipFile(f"{path}{permutation}.zip", "r") as ziph:
    # ziph is zipfile handle
    for filename in ziph.namelist():
        if filename.endswith(").json"):
            # Opening JSON file
            with ziph.open(filename, "r") as openfile:
                # Reading from JSON file 
                # https://stackoverflow.com/questions/37200302/
                # https://pythonspeed.com/articles/json-memory-streaming/
                # for n, item in enumerate(tqdm(ijson.items(openfile, "item"))):
                #    if n < howmany:
                #        items.append(item)
                #        # gc.collect()
                #    else:
                #        break
                items = json.load(openfile)
                if verbose:
                    print(openfile.name)

test
(2, 2).json


In [ ]:
items = loader(permutation, path)

In [20]:
if compile:
    # results = {}
    for permutation in [(12, 2)]:  # permutations:
        n_cars, n_moto = permutation
        results[str(permutation)] = {
            "VKT_cars": [],
            "VHT_cars": [],
            "VKT_moto": [],
            "VHT_moto": [],
        }
        items = loader(permutation, path)
        accidents = 0
        for item in items:
            if isinstance(item[0], list):
                (
                    VKT_cars,
                    VHT_cars,
                    VKT_moto,
                    VHT_moto,
                ) = aggregate(item[0], n_cars, n_moto)
                results[str(permutation)]["VKT_cars"].append(VKT_cars)
                results[str(permutation)]["VHT_cars"].append(VHT_cars)
                results[str(permutation)]["VKT_moto"].append(VKT_moto)
                results[str(permutation)]["VHT_moto"].append(VHT_moto)
            else:
                accidents += 1
        error = accidents / len(items)
        results[str(permutation)]["error"] = error
        if n_cars > 2 and n_moto > 2:
            percolation = percolate(items, n_moto, rng, start=5)
        else:
            percolation = None
        results[str(permutation)]["percolation"] = percolation
        items = None

(12, 2).json


In [8]:
write = compile
if write:
    with open(f"{path}results_.json", "w") as outfile:
        json.dump(results, outfile)